In [1]:
import torch
from d2l import torch as d2l

In [2]:
def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(X0, K0) for X0, K0 in zip(X, K))

In [3]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [4]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, K0) for K0 in K], 0)

In [5]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [6]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

维度是$1\times 1$的卷基层，相当于全连接层

In [7]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)

    return Y.reshape((c_o, h, w))

In [8]:
X = torch.normal(0, 1, (3, 3, 3))
X

tensor([[[ 0.6529,  0.2765,  1.8413],
         [-0.1131,  0.7430,  0.1493],
         [ 0.2261, -0.8534, -0.0125]],

        [[-0.1404, -0.4494,  0.1868],
         [-0.0331, -1.0246,  1.2810],
         [ 0.7074,  1.5114,  0.2218]],

        [[ 1.6395,  2.4965,  0.7941],
         [-2.3726, -0.2729,  0.8927],
         [-0.3166, -0.6565, -1.2168]]])

In [9]:
K = torch.normal(0, 1, (2, 3, 1, 1))
K

tensor([[[[-0.3178]],

         [[-0.4604]],

         [[ 0.0748]]],


        [[[-0.2861]],

         [[ 1.1589]],

         [[-0.0491]]]])

In [10]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y1

tensor([[[-0.0203,  0.3057, -0.6119],
         [-0.1262,  0.2152, -0.5705],
         [-0.4213, -0.4738, -0.1892]],

        [[-0.4300, -0.7225, -0.3493],
         [ 0.1105, -1.3866,  1.3980],
         [ 0.7707,  2.0280,  0.3204]]])

In [11]:
Y2 = corr2d_multi_in_out(X, K)
Y2

tensor([[[-0.0203,  0.3057, -0.6119],
         [-0.1262,  0.2152, -0.5705],
         [-0.4213, -0.4738, -0.1892]],

        [[-0.4300, -0.7225, -0.3493],
         [ 0.1105, -1.3866,  1.3980],
         [ 0.7707,  2.0280,  0.3204]]])

In [12]:
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6